In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import KFold
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv('./cleaned_datasets/toy_data.csv',index_col=0)  

In [3]:
df

,x0,x1,x2,x3,x4,x5,y
0,0.366540,0.878304,1.038823,1.103452,156.014503,0,1626.664038
1,0.034019,1.776411,-1.344630,1.495489,160.756809,0,1614.286202
2,-0.078950,1.258357,-1.045738,0.695931,163.798515,0,1589.071117
3,-0.829983,-0.327983,1.312429,0.779775,174.602188,1,1626.477706
4,0.775547,3.168103,-0.701222,1.635968,148.929220,0,1625.306675
...,...,...,...,...,...,...,...
199995,1.141587,5.224048,0.743879,1.874382,161.278138,0,1703.145237
199996,0.949447,4.448228,0.005072,0.155069,158.457684,0,1594.895812
199997,0.664946,3.340872,-0.568699,1.328317,161.181416,0,1617.162981
199998,0.538717,2.695474,-0.621813,1.137954,160.854668,0,1610.381437


In [4]:
df.drop(columns=['x1'])

,x0,x2,x3,x4,x5,y
0,0.366540,1.038823,1.103452,156.014503,0,1626.664038
1,0.034019,-1.344630,1.495489,160.756809,0,1614.286202
2,-0.078950,-1.045738,0.695931,163.798515,0,1589.071117
3,-0.829983,1.312429,0.779775,174.602188,1,1626.477706
4,0.775547,-0.701222,1.635968,148.929220,0,1625.306675
...,...,...,...,...,...,...
199995,1.141587,0.743879,1.874382,161.278138,0,1703.145237
199996,0.949447,0.005072,0.155069,158.457684,0,1594.895812
199997,0.664946,-0.568699,1.328317,161.181416,0,1617.162981
199998,0.538717,-0.621813,1.137954,160.854668,0,1610.381437


In [5]:
y = df.pop('y')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.9)
#subsampling beacuse so much data
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2)




In [6]:
X_train

,x0,x1,x2,x3,x4,x5
113814,1.318370,5.640943,0.407907,1.461615,160.227469,1
3724,0.156727,2.241118,-0.471432,1.292234,155.597346,0
106766,2.879562,6.286126,-1.290415,2.771577,162.717001,1
170421,0.586244,2.778591,0.630174,1.544564,162.513610,0
126161,1.046261,4.673772,0.556443,1.481107,163.899999,1
...,...,...,...,...,...,...
181633,0.574737,2.490747,-0.107254,0.987433,173.151079,1
150824,1.771976,5.901293,-0.363569,1.627214,153.883155,0
64076,1.178824,5.114977,-0.639217,1.729847,165.271862,0
36245,0.682461,2.787411,-0.800074,1.371049,159.767746,0


In [7]:
y_train

113814    1703.122934
3724      1619.000434
106766    1654.313531
170421    1658.141098
126161    1700.201051
             ...     
181633    1648.387948
150824    1636.598448
64076     1625.540344
36245     1614.925889
11494     1682.852592
Name: y, Length: 16000, dtype: float64

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
class DataFrameDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [10]:
def create_dataloader(X_train, X_test, y_train, y_test, batch_size=32):
    # Create custom Dataset objects for train and test data
    train_dataset = DataFrameDataset(X_train, y_train)
    test_dataset = DataFrameDataset(X_test, y_test)

    # Create DataLoaders for train and test data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [11]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_layers, hidden_units, dropout, dropout_array, input_dim, output_dim, activation=nn.ReLU(),norm=False):
        if hidden_layers != len(hidden_units):
            print("Error: wrong size of hidden_layers or hidden_units")
            return
        layers = []
        i = 0
        if norm:
            layers.append(nn.BatchNorm1d(input_dim))

        super(Net, self).__init__()
        
        layers.append(nn.Linear(input_dim, hidden_units[i]))
        layers.append(activation)
        if dropout:
            layers.append(nn.Dropout(dropout_array[i]))

        for _ in range(hidden_layers - 1):
            i += 1
            layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(activation)
            if dropout:
                layers.append(nn.Dropout(dropout_array[i]))

        layers.append(nn.Linear(hidden_units[-1], output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [12]:
model = Net(hidden_layers=3, hidden_units=[24, 24, 24], dropout=True, dropout_array=[0.2, 0.3, 0.4], input_dim=10, output_dim=1, activation=nn.Sigmoid(),norm=True)


In [13]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model.to(device)

In [14]:
summary(model,input_size=(10,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                   [-1, 10]              20
            Linear-2                   [-1, 24]             264
           Sigmoid-3                   [-1, 24]               0
           Dropout-4                   [-1, 24]               0
            Linear-5                   [-1, 24]             600
           Sigmoid-6                   [-1, 24]               0
           Dropout-7                   [-1, 24]               0
            Linear-8                   [-1, 24]             600
           Sigmoid-9                   [-1, 24]               0
          Dropout-10                   [-1, 24]               0
           Linear-11                    [-1, 1]              25
Total params: 1,509
Trainable params: 1,509
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [15]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [16]:
def train(config, checkpoint_dir=None):
    hidden_layers=config["hidden_layers"]
    hidden_units=config["hidden_units"]
    dropout=config["dropout"]
    dropout_array=config["dropout_array"]
    activation=config["activation"]
    norm=config["norm"]
    lr=config["lr"]
    input_dim=X_train.shape[1]
    output_dim=1
    
    train_ds = TensorDataset(torch.Tensor(X_train_scaled), torch.Tensor(y_train.to_numpy()))
    test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.to_numpy()))
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 0
    avg_loss = 0
    avg_r2 = 0

    for train_idx, val_idx in kfold.split(train_ds):
        print("This is fold :" ,fold)
        #trainloader, testloader = create_dataloader(X_train_scaled, X_test_scaled, y_train.values, y_test.values, batch_size=config["batch_size"])
        net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=input_dim, output_dim=output_dim, activation=activation,norm=norm)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)
        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])
        train_subset = torch.utils.data.Subset(train_ds, train_idx)
        val_subset = torch.utils.data.Subset(train_ds, val_idx)

        train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=config["batch_size"])
    
        for epoch in range(config["epochs"]):
            running_loss = 0.0
            train_loss=0.0
            val_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            train_loss=running_loss/len(train_loader)
               
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()      
            avg_loss += val_loss / len(val_loader)
            y_true = y_test.to_numpy()
            y_pred = net(torch.Tensor(X_test_scaled).to(device)).cpu()
            r2 = r2_score(y_true, y_pred)
            print(r2)
            tune.report(r2_curr=r2)
            avg_r2 += r2
        fold += 1
        

    avg_loss /= fold
    avg_r2 /= fold
    print("Average loss across folds: ", avg_loss)
    print("Average R2 score across folds: ", avg_r2)
    tune.report(loss=avg_loss,train_loss=train_loss,r2_score=avg_r2,r2_curr=r2)

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
device

'cuda'

In [19]:
def generate_hidden_units(hidden_layers):
    return np.random.choice(a=[16,32,64,128], size=hidden_layers).tolist()

search_space = {
    "batch_size": tune.choice([128,256,512, 1024, 2048]),
    #"batch_size": tune.choice([1024, 2048]),
    "lr": tune.loguniform(1e-7, 1e-2),
    "epochs": tune.choice([10,25,50,75]),
    #"epochs": tune.choice([5]),
    "hidden_layers": tune.randint(1, 4),
    "hidden_units": tune.sample_from(lambda spec: generate_hidden_units(spec.config.hidden_layers)),
    "dropout": tune.choice([True, False]),
    "dropout_array": tune.sample_from(lambda spec: np.random.uniform(low=0.1, high=0.8, size=spec.config.hidden_layers).tolist()),
    "activation": tune.choice([nn.ReLU()]),
    "norm": tune.choice([True, False])
}

In [20]:
scheduler = ASHAScheduler(metric="r2_curr", mode="max", grace_period=2, reduction_factor=2)
"""
pbt_scheduler = PopulationBasedTraining(
    time_attr='training_iteration',
    metric='loss',
    mode='min',
    perturbation_interval=1,
    hyperparam_mutations=search_space
)
"""
reporter = CLIReporter(metric_columns=["loss", "training_iteration","train_loss"])

In [21]:
analysis = tune.run(
        train,
        resources_per_trial={"cpu": 8, "gpu": 1 if torch.cuda.is_available() else 0},
        config=search_space,
        num_samples=100,
        scheduler=scheduler,
        progress_reporter=reporter)

print("Best hyperparameters found were: ", analysis.get_best_config(metric="r2_score", mode="max"))


2023-05-17 16:26:54,379	INFO worker.py:1538 -- Started a local Ray instance.
C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\ray\tune\trainable\function_trainable.py:610: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(


== Status ==
Current time: 2023-05-17 16:26:57 (running for 00:00:00.94)
Memory usage on this node: 11.1/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.56 GiB heap, 0.0/1.78 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-17_16-26-56
Number of trials: 17/100 (16 PENDING, 1 RUNNING)
+-------------------+----------+----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+
| Trial name        | status   | loc            | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |
|-------------------+----------+----------------+--------------+--------------+-----------+----------------------+----------+-----------------+---------

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,r2_curr,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_e0378_00000,2023-05-17_16-27-09,True,,75824aa625984da9a6bcb61046ce6dae,DESKTOP-8SILQJU,6,127.0.0.1,2868,-881.899,8.84271,0.00500989,8.84271,1684333629,0,,6,e0378_00000,0.00400448
train_e0378_00001,2023-05-17_16-27-47,True,,4e80753c61ca4cb9b6581727bca5a098,DESKTOP-8SILQJU,6,127.0.0.1,10784,-656.317,33.9341,0.00600576,33.9341,1684333667,0,,6,e0378_00001,0.00500488
train_e0378_00002,2023-05-17_16-28-31,True,,408314307650482c8bd732008abd22cc,DESKTOP-8SILQJU,6,127.0.0.1,2124,-601.264,39.4579,0.0055356,39.4579,1684333711,0,,6,e0378_00002,0.00556517
train_e0378_00003,2023-05-17_16-29-08,True,,c471c10ecc4049beb63c07822f841540,DESKTOP-8SILQJU,6,127.0.0.1,13160,-14.0831,32.5025,0.006006,32.5025,1684333748,0,,6,e0378_00003,0.00400376
train_e0378_00004,2023-05-17_16-29-33,True,,df8176fb994946c0883d7350896125c9,DESKTOP-8SILQJU,2,127.0.0.1,24856,-1032.89,20.8565,9.52601,20.8565,1684333773,0,,2,e0378_00004,0.00400472
train_e0378_00005,2023-05-17_16-29-50,True,,7d8e3a7ccb524339bc949c99af9db2f7,DESKTOP-8SILQJU,6,127.0.0.1,31012,0.887017,12.8777,0.00200176,12.8777,1684333790,0,,6,e0378_00005,0.00400257
train_e0378_00006,2023-05-17_16-30-59,True,,c008e7624a8841a583651065f01137db,DESKTOP-8SILQJU,6,127.0.0.1,18224,-6.96288,64.7542,0.00657248,64.7542,1684333859,0,,6,e0378_00006,0.00412774
train_e0378_00007,2023-05-17_16-31-08,True,,3a6f72ee59064935a6e8cc148c15e8e7,DESKTOP-8SILQJU,2,127.0.0.1,16604,-1032.82,4.4683,1.3002,4.4683,1684333868,0,,2,e0378_00007,0.00400376
train_e0378_00008,2023-05-17_16-31-16,True,,d762adbb9fa146d38d0fd7ba9b6bb47e,DESKTOP-8SILQJU,2,127.0.0.1,3820,-1032.96,4.39213,1.29111,4.39213,1684333876,0,,2,e0378_00008,0.00500417
train_e0378_00009,2023-05-17_16-31-34,True,,49d073e5644846b694f876a67245984c,DESKTOP-8SILQJU,2,127.0.0.1,2904,-1031.41,14.028,5.87392,14.028,1684333894,0,,2,e0378_00009,0.00500631


== Status ==
Current time: 2023-05-17 16:27:04 (running for 00:00:08.27)
Memory usage on this node: 13.1/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.56 GiB heap, 0.0/1.78 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-17_16-26-56
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status   | loc            | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   training_iteration |
|-------------------+----------+----------------+--------------+--------------+-----------+---------------

2023-05-17 16:27:09,692	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}


...

2023-05-17 17:11:50,501	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}


(train pid=26820) -980.8162183352069
(train pid=10940) This is fold : 0
== Status ==
Current time: 2023-05-17 17:11:56 (running for 00:45:00.13)
Memory usage on this node: 12.0/15.9 GiB 
Using AsyncHyperBand: num_stopped=74
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -108.9378388031475 | Iter 2.000: -1031.5491049946131
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.56 GiB heap, 0.0/1.78 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-17_16-26-56
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+-----------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr |

2023-05-17 17:12:00,306	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}
2023-05-17 17:12:00,466	INFO tune.py:762 -- Total run time: 2704.41 seconds (2704.18 seconds for the tuning loop).


== Status ==
Current time: 2023-05-17 17:12:00 (running for 00:45:04.20)
Memory usage on this node: 12.6/15.9 GiB 
Using AsyncHyperBand: num_stopped=75
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -108.9378388031475 | Iter 2.000: -1031.7244820319065
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/3.56 GiB heap, 0.0/1.78 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-17_16-26-56
Number of trials: 100/100 (100 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+------------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |       loss |   training_iteration |   train_loss |
|-------------------+---

Best hyperparameters found were:  {'batch_size': 128, 'lr': 0.0005963747684911029, 'epochs': 75, 'hidden_layers': 3, 'hidden_units': [64, 32, 32], 'dropout': False, 'dropout_array': [0.37506010026683856, 0.642260937333071, 0.6903269884596587], 'activation': ReLU(), 'norm': True}


In [22]:
analysis.results_df.sort_values(by=['r2_score', 'loss'],na_position='first').tail(10)

,loss,train_loss,r2_score,r2_curr,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,...,experiment_tag,config/batch_size,config/lr,config/epochs,config/hidden_layers,config/hidden_units,config/dropout,config/dropout_array,config/activation,config/norm
trial_id,,,,,,,,,,,,,,,,,,,,,
e0378_00006,150.514475,147.285390,-7.666790,-6.962879,0.006572,True,None,None,6,c008e7624a8841a583651065f01137db,...,"6_activation=ReLU,batch_size=128,dropout=True,...",128,0.005133,50,3,"[16, 128, 64]",True,"[0.34442218112664863, 0.28963242061053707, 0.1...",ReLU(),False
e0378_00033,80.458642,86.212924,-1.623327,-1.682604,0.012010,True,None,None,6,ca8259ec1b2745a8b4eee99e5e019261,...,"33_activation=ReLU,batch_size=128,dropout=Fals...",128,0.000373,50,1,[32],False,[0.7938220447954191],ReLU(),True
e0378_00091,61.788024,62.047740,0.588905,0.483576,0.003003,True,None,None,6,d737e127a7a340dba4e3bf5ccc516a2c,...,"91_activation=ReLU,batch_size=256,dropout=Fals...",256,0.001372,10,3,"[128, 128, 16]",False,"[0.4298569588359321, 0.37436295677544007, 0.22...",ReLU(),True
e0378_00087,21.314754,17.096869,0.804750,0.950651,0.023021,True,None,None,6,314482081c9e4cb7b0c7e788845fc088,...,"87_activation=ReLU,batch_size=256,dropout=Fals...",256,0.006690,75,3,"[128, 128, 64]",False,"[0.5492515679242129, 0.6978083262885771, 0.349...",ReLU(),True
e0378_00005,19.310717,18.487281,0.868494,0.887017,0.002002,True,None,None,6,7d8e3a7ccb524339bc949c99af9db2f7,...,"5_activation=ReLU,batch_size=128,dropout=False...",128,0.002618,10,1,[128],False,[0.7402627474796165],ReLU(),True
e0378_00028,17.863642,17.874686,0.882546,0.884139,0.011171,True,None,None,6,45c21bc1c50043fc809665e532d99247,...,"28_activation=ReLU,batch_size=512,dropout=Fals...",512,0.004358,25,1,[32],False,[0.4660323726270681],ReLU(),True
e0378_00052,17.343243,17.221784,0.893090,0.900809,0.015013,True,None,None,6,2bdacdcd05c346ca91c738ea28908050,...,"52_activation=ReLU,batch_size=256,dropout=Fals...",256,0.005056,50,1,[64],False,[0.5743670208331296],ReLU(),True
e0378_00071,9.984925,7.535250,0.959749,0.986025,0.004004,True,None,None,6,bbf850b7307e4eed988d39f2dfb58440,...,"71_activation=ReLU,batch_size=256,dropout=Fals...",256,0.008147,10,3,"[16, 64, 32]",False,"[0.6867636042986088, 0.5582569621759675, 0.526...",ReLU(),False
e0378_00015,10.074465,12.148007,0.960883,0.997194,0.008007,True,None,None,6,f393b094cba64df3a33baa08100e1363,...,"15_activation=ReLU,batch_size=128,dropout=Fals...",128,0.002970,75,3,"[128, 32, 32]",False,"[0.7543949710482903, 0.777876878942994, 0.1459...",ReLU(),True


In [23]:
cfg = {'batch_size': 128, 'lr': 0.000596, 'epochs': 75, 'hidden_layers': 3, 'hidden_units': [64, 32, 32], 'dropout': False, 'dropout_array': [0.375, 0.642, 0.690], 'activation': nn.ReLU(), 'norm': True}
hidden_layers=cfg["hidden_layers"]
hidden_units=cfg["hidden_units"]
dropout=cfg["dropout"]
dropout_array=cfg["dropout_array"]
activation=cfg["activation"]
norm=cfg["norm"]
lr=cfg["lr"]

In [24]:
net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=6, output_dim=1, activation=activation,norm=norm)

In [25]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
net = net.to(device)

In [26]:
summary(net,input_size=(6,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                    [-1, 6]              12
            Linear-2                   [-1, 64]             448
              ReLU-3                   [-1, 64]               0
            Linear-4                   [-1, 32]           2,080
              ReLU-5                   [-1, 32]               0
            Linear-6                   [-1, 32]           1,056
              ReLU-7                   [-1, 32]               0
            Linear-8                    [-1, 1]              33
Total params: 3,629
Trainable params: 3,629
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [69]:
best_config={'batch_size': 128, 'lr': 0.001104, 'epochs': 25, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.375, 0.642, 0.690], 'activation': nn.ReLU(), 'norm': False}

In [70]:
best_config

{'batch_size': 128,
 'lr': 0.001104,
 'epochs': 25,
 'hidden_layers': 3,
 'hidden_units': [128, 64, 32],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [31]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            print(r2_scores)
            print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    return avg_r2, avg_rmse
avg_r2, avg_rmse = cross_val_train(cfg)
print(f"Average R2 score: {avg_r2}")
print(f"Average RMSE: {avg_rmse}")


[0.9676414600004224]
[9.546536620051658]
[0.9676414600004224, 0.970492042494691]
[9.546536620051658, 9.155571530632512]
[0.9676414600004224, 0.970492042494691, 0.9565413047516705]
[9.546536620051658, 9.155571530632512, 10.449445121287411]
[0.9676414600004224, 0.970492042494691, 0.9565413047516705, 0.9527058086624547]
[9.546536620051658, 9.155571530632512, 10.449445121287411, 11.107651251762961]
[0.9676414600004224, 0.970492042494691, 0.9565413047516705, 0.9527058086624547, 0.9627973221774175]
[9.546536620051658, 9.155571530632512, 10.449445121287411, 11.107651251762961, 9.654703432775255]
Average R2 score: 0.9620355876173312
Average RMSE: 9.98278159130196


In [29]:
!pip install deap


Defaulting to user installation because normal site-packages is not writeable


In [72]:
best_config

{'batch_size': 128,
 'lr': 0.001104,
 'epochs': 25,
 'hidden_layers': 3,
 'hidden_units': [128, 64, 32],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [32]:
import numpy as np
from skopt import gp_minimize

fixed_config = {
    'hidden_layers': 3,
 'hidden_units': [64, 32, 32],
 'dropout': False,
 'dropout_array': [0.375, 0.642, 0.690],
 'activation': nn.ReLU(),
 'norm': True
}

# The objective function to minimize (negative R^2)
def objective(params):
    batch_size, lr, epochs = params
    config = {'batch_size': int(batch_size), 'lr': lr, 'epochs': int(epochs)}
    config.update(fixed_config)

    r2, rmse = cross_val_train(config)
    return rmse  # Minimize negative R^2 to maximize R^2

# Hyperparameter search space
space = [
    (25, 100),  # batch_size
    (1e-4, 1e-2, 'log-uniform'),  # lr
    (25, 100)  # epochs
]

# Bayesian optimization using Gaussian Process
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=50,  # Number of iterations
    verbose=True
)

# Print the optimized hyperparameters
print("Best hyperparameters found:")
print("Batch size:", result.x[0])
print("Learning rate:", result.x[1])
print("Epochs:", result.x[2])


Iteration No: 1 started. Evaluating function at random point.
[0.9822175416058837]
[7.0769688550975935]
[0.9822175416058837, 0.9842937525041999]
[7.0769688550975935, 6.679625341718572]
[0.9822175416058837, 0.9842937525041999, 0.9812300724019337]
[7.0769688550975935, 6.679625341718572, 6.8673026306586]
[0.9822175416058837, 0.9842937525041999, 0.9812300724019337, 0.9761178908138693]
[7.0769688550975935, 6.679625341718572, 6.8673026306586, 7.893228465930934]
[0.9822175416058837, 0.9842937525041999, 0.9812300724019337, 0.9761178908138693, 0.9721042543338216]
[7.0769688550975935, 6.679625341718572, 6.8673026306586, 7.893228465930934, 8.36028003415781]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 101.1107
Function value obtained: 7.3755
Current minimum: 7.3755
Iteration No: 2 started. Evaluating function at random point.
[0.98691861693786]
[6.069853585845265]
[0.98691861693786, 0.9359028025547035]
[6.069853585845265, 13.493835326309153]
[0.98691861693786, 0.93590280255

[0.9927500032658415, 0.9727057713774698, 0.9745262150892364, 0.9911807556305047]
[4.518770544737293, 8.805444778727615, 8.000214410595182, 4.796605729909272]
[0.9927500032658415, 0.9727057713774698, 0.9745262150892364, 0.9911807556305047, 0.9107343676978022]
[4.518770544737293, 8.805444778727615, 8.000214410595182, 4.796605729909272, 14.955260426960859]
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 398.8077
Function value obtained: 8.2153
Current minimum: 5.1142
Iteration No: 12 started. Searching for the next optimal point.
[0.9910390318867716]
[5.023759606597028]
[0.9910390318867716, 0.9828822618544899]
[5.023759606597028, 6.973311643519259]
[0.9910390318867716, 0.9828822618544899, 0.9917607749856922]
[5.023759606597028, 6.973311643519259, 4.5498603775439035]
[0.9910390318867716, 0.9828822618544899, 0.9917607749856922, 0.9707716118731056]
[5.023759606597028, 6.973311643519259, 4.5498603775439035, 8.732141538719858]
[0.9910390318867716, 0.982882261854

[0.9819225020582193, 0.9881536568216684, 0.9559487908348793, 0.9813967212410021, 0.9780517382443711]
[7.135436475156362, 5.8010691204465, 10.52043759827474, 6.966474961718588, 7.415695740897462]
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 188.6968
Function value obtained: 7.5678
Current minimum: 5.1142
Iteration No: 22 started. Searching for the next optimal point.
[0.993439874352082]
[4.298406284445233]
[0.993439874352082, 0.9837093623218797]
[4.298406284445233, 6.802756526475727]
[0.993439874352082, 0.9837093623218797, 0.9656002883581769]
[4.298406284445233, 6.802756526475727, 9.29677432518063]
[0.993439874352082, 0.9837093623218797, 0.9656002883581769, 0.9744020243892034]
[4.298406284445233, 6.802756526475727, 9.29677432518063, 8.171864260651928]
[0.993439874352082, 0.9837093623218797, 0.9656002883581769, 0.9744020243892034, 0.9952110053354869]
[4.298406284445233, 6.802756526475727, 9.29677432518063, 8.171864260651928, 3.463969136968414]
Iteration

Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 191.6986
Function value obtained: 6.9109
Current minimum: 5.1142
Iteration No: 32 started. Searching for the next optimal point.
[0.9672935121198485]
[9.597725817860269]
[0.9672935121198485, 0.9724062167224958]
[9.597725817860269, 8.853632881399566]
[0.9672935121198485, 0.9724062167224958, 0.9752897678612519]
[9.597725817860269, 8.853632881399566, 7.879402763481106]
[0.9672935121198485, 0.9724062167224958, 0.9752897678612519, 0.9562773956012015]
[9.597725817860269, 8.853632881399566, 7.879402763481106, 10.680002279211266]
[0.9672935121198485, 0.9724062167224958, 0.9752897678612519, 0.9562773956012015, 0.964462211564169]
[9.597725817860269, 8.853632881399566, 7.879402763481106, 10.680002279211266, 9.436197804244262]
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 179.4820
Function value obtained: 9.2894
Current minimum: 5.1142
Iteration No: 33 started. Searching for the next o

[0.9768025260258631]
[8.082984235521588]
[0.9768025260258631, 0.9921766508561903]
[8.082984235521588, 4.714246004261563]
[0.9768025260258631, 0.9921766508561903, 0.9851482779292862]
[8.082984235521588, 4.714246004261563, 6.108622598209981]
[0.9768025260258631, 0.9921766508561903, 0.9851482779292862, 0.9574101739977245]
[8.082984235521588, 4.714246004261563, 6.108622598209981, 10.540744016784712]
[0.9768025260258631, 0.9921766508561903, 0.9851482779292862, 0.9574101739977245, 0.9935059469240471]
[8.082984235521588, 4.714246004261563, 6.108622598209981, 10.540744016784712, 4.033757205749332]
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 532.2935
Function value obtained: 6.6961
Current minimum: 4.5058
Iteration No: 43 started. Searching for the next optimal point.
[0.9923742092476673]
[4.6344033119053885]
[0.9923742092476673, 0.9916756796766228]
[4.6344033119053885, 4.862843323461881]
[0.9923742092476673, 0.9916756796766228, 0.9885773709794293]
[4.6344033

C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[0.981225955067056]
[7.271605512160674]
[0.981225955067056, 0.8760780226681815]
[7.271605512160674, 18.762471804900503]
[0.981225955067056, 0.8760780226681815, 0.9825227709661287]
[7.271605512160674, 18.762471804900503, 6.626606414273313]
[0.981225955067056, 0.8760780226681815, 0.9825227709661287, 0.8491399947056434]
[7.271605512160674, 18.762471804900503, 6.626606414273313, 19.838342915267255]
[0.981225955067056, 0.8760780226681815, 0.9825227709661287, 0.8491399947056434, 0.624274374406209]
[7.271605512160674, 18.762471804900503, 6.626606414273313, 19.838342915267255, 30.6822529093388]
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 162.0803
Function value obtained: 16.6363
Current minimum: 4.5058
Iteration No: 45 started. Searching for the next optimal point.
[0.9813039236224017]
[7.256490323968286]
[0.9813039236224017, 0.9899694598748181]
[7.256490323968286, 5.337993555617055]
[0.9813039236224017, 0.9899694598748181, 0.988197872178719]
[7.256490323968

In [74]:
best_config_final={'batch_size': 25, 'lr': 0.0004990515469382572, 'epochs': 100, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.375, 0.642, 0.690], 'activation': nn.ReLU(), 'norm': False}